In [4]:
import os
import tensorflow as tf
import numpy as np
import math
from PIL import Image
import csv
from pathlib import Path
import cv2

blue = (255,0,0)
green = (0,255,0)
red = (0,0,255)
white = (255,255,255)
img = np.zeros((800,800,3), np.uint8)

In [5]:
HOME_PATH = '/home/cu'
DATA_PATH = os.path.join(HOME_PATH, 'darknet/train_img/original_img')
SAVE_PATH = os.path.join(HOME_PATH, 'darknet/train_img')

In [6]:
file_list = os.listdir(DATA_PATH)
img_list = [file for file in file_list if file.endswith('.jpg')]
txt_list = [file for file in file_list if file.endswith('.txt')]
img_list

degree = 10
range_max = 36

for i in range(0, len(txt_list)):
    #image file
    f = open(os.path.join(DATA_PATH, txt_list[i]))
    original_img = Image.open(f.name.replace('txt', 'jpg'))
    img_names = ['{0}{1}.jpg'.format(f.name.replace('original_img/', '').replace('.txt', '_'), i * degree) for i in range(0, range_max)] 
    
    #txt file
    txt_names = [img_names[i].replace('jpg', 'txt') for i in range(0, range_max)]

    #(현재 한줄만 읽음)    
    data = f.readline().split()
    img_class = data[0]
    x = float(data[1])
    y = float(data[2])    
    w = float(data[3])
    h = float(data[4])
    
    x0 = 0.5
    y0 = 0.5

    x1 = x - (w/2)
    y1 = y - (h/2)
    
    x2 = x + (w/2)
    y2 = y - (h/2)
    
    x3 = x - (w/2)
    y3 = y + (h/2)
    
    x4 = x + (w/2)
    y4 = y + (h/2)
    
    for j in range(0, 36):
        #Image 처리
        rotate_img = original_img.rotate(j * 10)
        rotate_img.save(img_names[j])        
        #print(txt_names[j])
        
        #bounding box
        theta =  math.radians(j * 10)
        #theta =  j * 10
                
        rx1 = x0 + (x1-x0) * math.cos(theta) - (y1-y0) * math.sin(theta)
        ry1 = y0 + (x1-x0) * math.sin(theta) + (y1-y0) * math.cos(theta)
        
        rx2 = x0 + (x2-x0) * math.cos(theta) - (y2-y0) * math.sin(theta)
        ry2 = y0 + (x2-x0) * math.sin(theta) + (y2-y0) * math.cos(theta)
        
        rx3 = x0 + (x3-x0) * math.cos(theta) - (y3-y0) * math.sin(theta)
        ry3 = y0 + (x3-x0) * math.sin(theta) + (y3-y0) * math.cos(theta)
        
        rx4 = x0 + (x4-x0) * math.cos(theta) - (y4-y0) * math.sin(theta)
        ry4 = y0 + (x4-x0) * math.sin(theta) + (y4-y0) * math.cos(theta)

        rX = [rx1, rx2, rx3, rx4]
        rY = [ry1, ry2, ry3, ry4]

        min_rx = min(rX)
        max_rx = max(rX)
        
        min_ry = min(rY)
        max_ry = max(rY)

        rW = max_rx - min_rx
        rH = max_ry - min_ry
        
        rx0 = min_rx + (rW/2)
        ry0 = min_ry + (rH/2)

        f2 = open(os.path.join(SAVE_PATH, txt_names[j]), 'w')
        f_data = str(img_class)+' '+str(rx0)+' '+str(ry0)+' '+str(rW)+' '+str(rH)
        f2.write(f_data)
        f2.close
        #print(rx0, ry0, rW, rH)
        #print(min_ry, max_ry)
        #print(theta)
    f.close()

In [46]:
print(img_class, x0, y0, w, h)

2 0.493359 0.288194 0.239844 0.240278


In [40]:
f_data

'2 0.515234 0.317361 0.3793668061888944 0.2617064874659112'